# Application Review

### 225HP @ 2200

In [ ]:
import pandas as pd
from nptdms import tdms
import matplotlib.pyplot as plt
import dtdms
import wpphist
%matplotlib inline

### Path to TDMS file:

In [ ]:
tdmspath = "./aof_seal_newguard_CAC.tdms"

### Constants from Performance Curve

In [ ]:
perf_curve_cac_min = 48  #degrees C
perf_curve_cac_max = 56  #degrees C
perf_curve_max_top_tank = 113 #degrees C

perf_curve_min_water_pump_inlet_95C = 103 #kPa
perf_curve_max_intake_restriction = -3.75 #kPa

### Setup Data Structures and Perform Standard calculations

In [ ]:
#create an instance of tdms data from the specified TDMS file
tdms = dtdms.drivetdms(tdmspath)

In [ ]:
#create base PANDAS data frame from subset of TDMS data in the dtdms class data_dict using pandas(dataframe from dict)
#more CAN data is available and can be enabled by editing the dtdms class.  All Temp and Pressure data will be available.
df = pd.DataFrame(tdms.data_dict)
df.columns.values

In [ ]:
#Add calculated columns to the base data frame
#df = pd.DataFrame(mL, index=mL['Timestamp'])

#Standard Calculations
df['Ambient'] = (df.tT01_Ambient_Air_01 + df.tT02_Ambient_Air_02) / 2.0
df['CorrectedCAC'] = 25 - df.Ambient + df.cCACOutTemp
df['LAT'] = perf_curve_max_top_tank - df.cCoolantTemp + df.Ambient
df['IntakeTempRise'] = df.tT03_Air_Intake - df.Ambient
df['IntakeRestrictionAvailable'] = df.pP01_Intake_Air_Restriction - perf_curve_max_intake_restriction

#Test Specific Calculations


### Plot overall Run.  Look at index at bottom for more detailed plot in next cell.

In [ ]:
df.plot(figsize=(12,24), color=('r'), subplots=True );

### Place the start and end timestamp of the test to be analyzed below

Not all data in the data frame should go into the CAC and temp rise histograms.  Specify only the data where the engine has been warmed up and is running under a representative load for this subset of the data.  Full test runs can be obtained by referening the df data frame instead of this test_subset data frame

In [ ]:
test_subset = df[800:1550]
section_plot = test_subset.plot(figsize=(15,30), color = ('g'), subplots=True);

# Histograms from the test_subset data

In [ ]:
wpphist.plot(data=test_subset['IntakeTempRise'].values, ylabel='Count', xlabel='Temperature (degC)', 
             title='Intake Temp Rise (degC) Histogram', color='green')

In [ ]:
wpphist.plot(data=test_subset['IntakeRestrictionAvailable'].values, ylabel='Count', xlabel='Pressure (kPa)', 
             title='Intake Restriction Margin (kPa) Histogram', color='pink')

In [ ]:
wpphist.plot(data=test_subset['CorrectedCAC'].values, ylabel='Count', xlabel='Temperature (degC)', 
             title='Corrected CAC (degC) Histogram', color='blue')

In [ ]:
wpphist.plot(data=test_subset['LAT'].values, ylabel='Count', xlabel='Temperature (degC)', 
             title='Limiting Ambient Temperature (degC) Histogram', color='orange')

In [ ]:
#wpphist.plot(data=test_subset['ACTempRise'].values, ylabel='Count', xlabel='Temperature (degC)', 
#             title='Hydraulic Cooler Temp Rise (degC) Histogram', color='black')

In [ ]:
#water_pump_df = test_subset[['pP04_Water_Pump_Inlet', 'cCoolantTemp']]
#wp_df = (water_pump_df[water_pump_df['cCoolantTemp'] > 82]) #filter out low coolant temp values

#wpphist.plot(data=wp_df['pP04_Water_Pump_Inlet'].values, ylabel='Count', xlabel='Pressure (kPa)', 
#             title='Water Pump Inlet Pressure (kPa) Histogram', color='yellow')
#wpphist.plot(data=test_subset['pP04_Water_Pump_Inlet'].values, ylabel='Count', xlabel='Pressure (kPa)', 
#             title='Water Pump Inlet Pressure (kPa) Histogram', color='yellow')
